In [1]:
import composuite
from diffusion.utils import *
import numpy as np
import matplotlib.pyplot as plt

def compute_error(agent_dataset, synthetic_dataset):
    
    agent_observations = agent_dataset['observations']
    random_indices = np.random.choice(agent_observations.shape[0], 1000, replace=False)
    sampled_agent_observations = agent_observations[random_indices]

    synthetic_observations = synthetic_dataset['observations']
    random_indices = np.random.choice(synthetic_observations.shape[0], 1000, replace=False)
    sampled_synthetic_observations = synthetic_observations[random_indices]

    mean_agent = np.mean(sampled_agent_observations, axis=0)
    mean_synthetic = np.mean(sampled_synthetic_observations, axis=0)

    error = np.linalg.norm(mean_agent - mean_synthetic)
    
    return error

In [2]:
base_agent_data_path = '/Users/shubhankar/Developer/compositional-rl-synth-data/data'
dataset_type = 'expert'
base_synthetic_data_path = '/Users/shubhankar/Developer/compositional-rl-synth-data/cluster_results/diffusion'

In [ ]:
representative_env = composuite.make('IIWA', 'Plate', 'ObjectWall', 'Push', use_task_id_obs=True, ignore_done=False)

In [4]:
runs = ['cond_diff_17', 'cond_diff_7', 'cond_diff_8', 
        'cond_diff_10', 'cond_diff_15', 'cond_diff_19', 
        'cond_diff_18', 'cond_diff_20', 'cond_diff_21']
num_train_tasks = [16, 32, 48, 64, 80, 96, 128, 176, 244]

In [ ]:
all_train_error_means = []
all_train_error_stds = []
all_test_error_means = []
all_test_error_stds = []

for run in tqdm(runs, desc='Run'):
    train_tasks = [
        task for task in os.listdir(os.path.join(base_synthetic_data_path, run, 'train'))
        if not task.startswith('.')
    ]
    errors = []
    for task in train_tasks:
        robot, obj, obst, subtask = task.split('_')
        agent_dataset = load_single_composuite_dataset(base_path=base_agent_data_path, 
                                                       dataset_type='expert', 
                                                       robot=robot, obj=obj, 
                                                       obst=obst, task=subtask)
        agent_dataset = transitions_dataset(agent_dataset)
        agent_dataset, _ = remove_indicator_vectors(representative_env.modality_dims, agent_dataset)

        synthetic_dataset = load_single_synthetic_dataset(base_path=os.path.join(base_synthetic_data_path, run, 'train'),
                                                        robot=robot, obj=obj, 
                                                        obst=obst, task=subtask)
        errors.append(compute_error(agent_dataset, synthetic_dataset))
    all_train_error_means.append(np.mean(errors))
    all_train_error_stds.append(np.std(errors))

    test_tasks = [
        task for task in os.listdir(os.path.join(base_synthetic_data_path, run, 'test'))
        if not task.startswith('.')
    ]
    errors = []
    for task in test_tasks:
        robot, obj, obst, subtask = task.split('_')
        agent_dataset = load_single_composuite_dataset(base_path=base_agent_data_path, 
                                                       dataset_type='expert', 
                                                       robot=robot, obj=obj, 
                                                       obst=obst, task=subtask)
        agent_dataset = transitions_dataset(agent_dataset)
        agent_dataset, _ = remove_indicator_vectors(representative_env.modality_dims, agent_dataset)

        synthetic_dataset = load_single_synthetic_dataset(base_path=os.path.join(base_synthetic_data_path, run, 'test'),
                                                        robot=robot, obj=obj, 
                                                        obst=obst, task=subtask)
        errors.append(compute_error(agent_dataset, synthetic_dataset))
    all_test_error_means.append(np.mean(errors))
    all_test_error_stds.append(np.std(errors))

In [ ]:
# plt.figure(figsize=(8, 6))
# plt.plot(num_train_tasks, all_train_error_means, label='Train Error', marker='o')
# plt.plot(num_train_tasks, all_test_error_means, label='Test Error', marker='s')

# plt.xlabel('Number of Training Tasks')
# plt.ylabel('Generalization Error')
# plt.title('Errors vs. Number of Training Tasks')
# plt.legend()
# plt.grid(True)

# plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

plt.errorbar(
    num_train_tasks, all_train_error_means, yerr=all_train_error_stds, 
    label='Train Error', marker='o', capsize=5, linestyle='-'
)
plt.errorbar(
    num_train_tasks, all_test_error_means, yerr=all_test_error_stds, 
    label='Test Error', marker='s', capsize=5, linestyle='--'
)

plt.xlabel('Number of Training Tasks')
plt.ylabel('Generalization Error')
plt.title('Errors vs. Number of Training Tasks')
plt.legend()
plt.grid(True)

plt.show()
